# 金融 证券 图谱搭建

对于，图谱的更新我们采取 neo4j APOC 中提供的功能——`apoc.periodic.iterate` 过程，批量更新图谱。

具体用法可参考下面的网址。

网址：https://neo4j.com/docs/labs/apoc/3.5/graph-updates/periodic-execution/

在这里，我们尽可能覆盖实际业务中更新场景

- 节点
    - 节点标签 labels （新增/删除）
    - 节点属性 properities（新增/修改）

- 关系
    - 关系 （新增）
    - 关系属性 properities（新增/修改

## 更新图谱

In [80]:
import os
from py2neo import Graph
import warnings
warnings.filterwarnings('ignore')

In [81]:
def check_result(result):
    for c in result :
        print(c)

In [82]:
graph = Graph(
    "http://localhost:7474",
    usernmae="neo4j",
    password="Neo4j"
)

### 更新节点：公司

In [83]:
query_node_company = """
CALL apoc.periodic.iterate(
    "LOAD CSV WITH HEADERS FROM 'file:///media/kevin/storage1/finance_graph/data_20200204/gap/node_companies.csv' AS row return row",
    "with row.fullname as company,
          row.name as short_name,
          row.symbol as symbol,
          row.market as market,
          row.exchange as exchange,
          row.list_status as list_status,
          row.list_date as list_date,
          row.delist_date as delist_date,
          row.setup_date as setup_date,
          row.label as label
    MERGE (a:COMPANY {company:company})
    SET a.short_name = (CASE WHEN short_name <> 'NotChange' THEN short_name ELSE a.short_name END),
        a.symbol = (CASE WHEN symbol <> 'NotChange' THEN symbol ELSE a.symbol END),
        a.market = (CASE WHEN market <> 'NotChange' THEN market ELSE a.market END),
        a.exchange = (CASE WHEN exchange <> 'NotChange' THEN exchange ELSE a.exchange  END),
        a.list_status = (CASE WHEN list_status <> 'NotChange' THEN list_status ELSE a.list_status END),
        a.list_date = (CASE WHEN list_date <> 'NotChange' THEN toFloat(list_date) ELSE a.list_date  END),
        a.delist_date = (CASE WHEN delist_date <> 'NotChange' THEN toFloat(delist_date) ELSE a.delist_date END),
        a.setup_date = (CASE WHEN setup_date <> 'NotChange' THEN setup_date ELSE a.setup_date END)
    REMOVE a:LISTED_COMPANY:FUND_CUSTODIAN:FUND_MANAGER    
    FOREACH(t in CASE WHEN 'LISTED_COMPANY' in split(label,';') THEN 1 ELSE null END | SET a:LISTED_COMPANY )
    FOREACH(t in CASE WHEN 'FUND_CUSTODIAN' in split(label,';') THEN 1 ELSE null END | SET a:FUND_CUSTODIAN )
    FOREACH(t in CASE WHEN 'FUND_MANAGER' in split(label,';') THEN 1 ELSE null END | SET a:FUND_MANAGER )
    RETURN a
    ",
    {batchSize:10000, parallel:true})
    """

In [84]:
result = graph.run(query_node_company)

In [85]:
check_result(result)

<Record batches=1 total=49 timeTaken=2 committedOperations=49 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 1, 'committed': 1, 'failed': 0, 'errors': {}} operations={'total': 49, 'committed': 49, 'failed': 0, 'errors': {}} wasTerminated=False>


### 更新节点：行业

In [86]:
query_node_industries = """
CALL apoc.periodic.iterate(
    "LOAD CSV WITH HEADERS FROM 'file:///media/kevin/storage1/finance_graph/data_20200204/gap/node_industries.csv' AS row return row",
    "with row.industry as industry,
          row.label as label
    MERGE (a:INDUSTRY {industry:industry})
    RETURN a
    ",
    {batchSize:10000, parallel:true})
    """

In [87]:
result = graph.run(query_node_industries)

In [88]:
check_result(result)

<Record batches=1 total=0 timeTaken=0 committedOperations=0 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 1, 'committed': 1, 'failed': 0, 'errors': {}} operations={'total': 0, 'committed': 0, 'failed': 0, 'errors': {}} wasTerminated=False>


### 更新节点：城市

In [89]:
query_node_city = """
CALL apoc.periodic.iterate(
    "LOAD CSV WITH HEADERS FROM 'file:///media/kevin/storage1/finance_graph/data_20200204/gap/node_city.csv' AS row return row",
    "with row.city as city,
          row.label as label
    MERGE (a:CITY {city:city})
    RETURN a
    ",
    {batchSize:10000, parallel:true})
    """

In [90]:
result = graph.run(query_node_city)

In [91]:
check_result(result)

<Record batches=1 total=0 timeTaken=0 committedOperations=0 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 1, 'committed': 1, 'failed': 0, 'errors': {}} operations={'total': 0, 'committed': 0, 'failed': 0, 'errors': {}} wasTerminated=False>


### 更新节点：省份

In [92]:
query_node_province = """
CALL apoc.periodic.iterate(
    "LOAD CSV WITH HEADERS FROM 'file:///media/kevin/storage1/finance_graph/data_20200204/gap/node_province.csv' AS row return row",
    "with row.province as province,
          row.label as label
    MERGE (a:PROVINCE {province:province})
    RETURN a
    ",
    {batchSize:10000, parallel:true})
    """

In [93]:
result = graph.run(query_node_province)

In [94]:
check_result(result)

<Record batches=1 total=0 timeTaken=0 committedOperations=0 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 1, 'committed': 1, 'failed': 0, 'errors': {}} operations={'total': 0, 'committed': 0, 'failed': 0, 'errors': {}} wasTerminated=False>


### 更新节点：基金

In [95]:
query_node_funds = """
CALL apoc.periodic.iterate(
    "LOAD CSV WITH HEADERS FROM 'file:///media/kevin/storage1/finance_graph/data_20200204/gap/node_funds.csv' AS row return row",
    "with row.fund as fund_code,
          row.name as name,
          row.fund_type as fund_type,
          row.invest_type as invest_type,
          row.type as type,
          row.benchmark as benchmark,
          row.market as market,
          row.found_date as found_date,
          row.delist_date as delist_date,
          row.status as status,
          row.label as label
    MERGE (a:FUND {fund_code:fund_code})
    SET a.name = (CASE WHEN name <> 'NotChange' THEN name ELSE a.name END),
        a.fund_type = (CASE WHEN fund_type <> 'NotChange' THEN fund_type ELSE a.fund_type END),
        a.invest_type = (CASE WHEN invest_type <> 'NotChange' THEN invest_type ELSE a.invest_type END),
        a.type = (CASE WHEN type <> 'NotChange' THEN type ELSE a.type  END),
        a.benchmark = (CASE WHEN benchmark <> 'NotChange' THEN benchmark ELSE a.benchmark END),
        a.market = (CASE WHEN market <> 'NotChange' THEN market ELSE a.market END),
        a.found_date = (CASE WHEN found_date <> 'NotChange' THEN toFloat(found_date) ELSE a.found_date  END),
        a.delist_date = (CASE WHEN delist_date <> 'NotChange' THEN toFloat(delist_date) ELSE a.delist_date  END),
        a.status = (CASE WHEN status <> 'NotChange' THEN status ELSE a.status END)
    RETURN a
    ",
    {batchSize:10000, parallel:true})
    """

In [96]:
result = graph.run(query_node_funds)

In [97]:
check_result(result)

<Record batches=1 total=378 timeTaken=3 committedOperations=378 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 1, 'committed': 1, 'failed': 0, 'errors': {}} operations={'total': 378, 'committed': 378, 'failed': 0, 'errors': {}} wasTerminated=False>


### 更新节点：人

In [98]:
query_node_managers = """
CALL apoc.periodic.iterate(
    "LOAD CSV WITH HEADERS FROM 'file:///media/kevin/storage1/finance_graph/data_20200204/gap/node_managers.csv' AS row return row",
    "with row.hash_cust as manager_id,
          row.name as name,
          row.gender as gender,
          row.edu as edu,
          row.national as national,
          row.birthday as birthday,
          row.label as label
    MERGE (a:MANAGER {manager_id:manager_id})
    SET a.name = (CASE WHEN name <> 'NotChange' THEN name ELSE a.name END),
        a.gender = (CASE WHEN gender <> 'NotChange' THEN gender ELSE a.gender END),
        a.edu = (CASE WHEN edu <> 'NotChange' THEN edu ELSE a.edu END),
        a.national = (CASE WHEN national <> 'NotChange' THEN national ELSE a.national  END),
        a.birthday = (CASE WHEN birthday <> 'NotChange' THEN toFloat(birthday) ELSE a.birthday END)
    RETURN a
    ",
    {batchSize:10000, parallel:true})
    """

In [99]:
result = graph.run(query_node_managers)

In [100]:
check_result(result)

<Record batches=1 total=0 timeTaken=0 committedOperations=0 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 1, 'committed': 1, 'failed': 0, 'errors': {}} operations={'total': 0, 'committed': 0, 'failed': 0, 'errors': {}} wasTerminated=False>


### 更新关系：城市-->省份

In [101]:
query_rel_city_in_province = """
CALL apoc.periodic.iterate(
    "LOAD CSV WITH HEADERS FROM 'file:///media/kevin/storage1/finance_graph/data_20200204/gap/rel_city_in_province.csv' AS row return row",
    "with row.city as city,
          row.province as province,
          row.type as type
    MERGE (a:CITY {city:city})
    MERGE (b:PROVINCE {province:province})
    MERGE (a)-[r:IN_PROVINCE]->(b)
    RETURN a,b,r
    ",
    {batchSize:10000, parallel:true})
    """

In [102]:
result = graph.run(query_rel_city_in_province)

In [103]:
check_result(result)

<Record batches=1 total=0 timeTaken=0 committedOperations=0 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 1, 'committed': 1, 'failed': 0, 'errors': {}} operations={'total': 0, 'committed': 0, 'failed': 0, 'errors': {}} wasTerminated=False>


### 更新关系：公司-->城市

In [104]:
query_rel_company_in_city = """
CALL apoc.periodic.iterate(
    "LOAD CSV WITH HEADERS FROM 'file:///media/kevin/storage1/finance_graph/data_20200204/gap/rel_company_in_city.csv' AS row return row",
    "with row.fullname as company,
          row.city as city,
          row.type as type
    MERGE (a:COMPANY {company:company})          
    MERGE (b:CITY {city:city})
    MERGE (a)-[r:IN_CITY]->(b)
    RETURN a,b,r
    ",
    {batchSize:10000, parallel:true})
    """

In [105]:
result = graph.run(query_rel_company_in_city)

In [106]:
check_result(result)

<Record batches=1 total=0 timeTaken=0 committedOperations=0 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 1, 'committed': 1, 'failed': 0, 'errors': {}} operations={'total': 0, 'committed': 0, 'failed': 0, 'errors': {}} wasTerminated=False>


### 更新关系：股票-->行业

In [107]:
query_rel_share_in_industry = """
CALL apoc.periodic.iterate(
    "LOAD CSV WITH HEADERS FROM 'file:///media/kevin/storage1/finance_graph/data_20200204/gap/rel_share_in_industry.csv' AS row return row",
    "with row.fullname as company,
          row.industry as industry,
          row.type as type
    MERGE (a:COMPANY {company:company})          
    MERGE (b:INDUSTRY {industry:industry})
    MERGE (a)-[r:IN_INDUSTRY]->(b)
    RETURN a,b,r
    ",
    {batchSize:10000, parallel:true})
    """

In [108]:
result = graph.run(query_rel_share_in_industry)

In [109]:
check_result(result)

<Record batches=1 total=0 timeTaken=0 committedOperations=0 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 1, 'committed': 1, 'failed': 0, 'errors': {}} operations={'total': 0, 'committed': 0, 'failed': 0, 'errors': {}} wasTerminated=False>


### 更新关系：公司（上市公司）--> 人(董事高管)

对于 上市公司 高管情况，其实有另外一种 处理方式，就是参照上面的处理方式把 6 类关系分别拆开用不同的文件进行更新。
这里没有做拆分，选择用 `cypher` 判断关系类型更新，速度稍慢。

In [113]:
query_rel_listed_company_has_manager = """
CALL apoc.periodic.iterate(
    "LOAD CSV WITH HEADERS FROM 'file:///media/kevin/storage1/finance_graph/data_20200204/gap/rel_listed_company_has_manager.csv' AS row return row",
    "with row.fullname as company,
          row.hash_cust as manager_id,
          row.begin_date as begin_date,
          row.end_date as end_date,
          row.title as title,
          row.lev as type
    MERGE (a:COMPANY {company:company})          
    MERGE (b:MANAGER {manager_id:manager_id})
    FOREACH(t in CASE WHEN type = '董事会成员' THEN 1 ELSE null END | MERGE (a)-[r:董事会成员 {title:title,begin_date:toFloat(begin_date)}]->(b) 
                                                                    SET r.end_date = (CASE WHEN end_date <> 'NotChange' THEN toFloat(end_date) ELSE r.end_date END)
                                                                    )
                                                                    
    FOREACH(t in CASE WHEN type = '高管' THEN 1 ELSE null END | MERGE (a)-[r:高管 {title:title,begin_date:toFloat(begin_date)}]->(b) 
                                                                    SET r.end_date = (CASE WHEN end_date <> 'NotChange' THEN toFloat(end_date) ELSE r.end_date END)
                                                                    )
    FOREACH(t in CASE WHEN type = '委员会成员' THEN 1 ELSE null END | MERGE (a)-[r:委员会成员 {title:title,begin_date:toFloat(begin_date)}]->(b) 
                                                                    SET r.end_date = (CASE WHEN end_date <> 'NotChange' THEN toFloat(end_date) ELSE r.end_date END)
                                                                    )
    FOREACH(t in CASE WHEN type = '监事' THEN 1 ELSE null END | MERGE (a)-[r:监事 {title:title,begin_date:toFloat(begin_date)}]->(b) 
                                                                    SET r.end_date = (CASE WHEN end_date <> 'NotChange' THEN toFloat(end_date) ELSE r.end_date END)
                                                                    )
    FOREACH(t in CASE WHEN type = '核心技术人员' THEN 1 ELSE null END | MERGE (a)-[r:核心技术人员 {title:title,begin_date:toFloat(begin_date)}]->(b) 
                                                                    SET r.end_date = (CASE WHEN end_date <> 'NotChange' THEN toFloat(end_date) ELSE r.end_date END)
                                                                    )
    FOREACH(t in CASE WHEN type = '其他' THEN 1 ELSE null END | MERGE (a)-[r:其他 {title:title,begin_date:toFloat(begin_date)}]->(b) 
                                                                    SET r.end_date = (CASE WHEN end_date <> 'NotChange' THEN toFloat(end_date) ELSE r.end_date END)
                                                                    )
    RETURN a,b
    ",
    {batchSize:1000, parallel:false})
    """

In [114]:
# 在关系匹配时，先对图数据库， 添加索引，提高更新效率
result = graph.run(query_rel_listed_company_has_manager)

In [115]:
check_result(result)

<Record batches=2 total=1251 timeTaken=0 committedOperations=1251 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 2, 'committed': 2, 'failed': 0, 'errors': {}} operations={'total': 1251, 'committed': 1251, 'failed': 0, 'errors': {}} wasTerminated=False>


### 更新关系：公募基金 --> 托管人

In [116]:
query_rel_fund_has_custodian = """
CALL apoc.periodic.iterate(
    "LOAD CSV WITH HEADERS FROM 'file:///media/kevin/storage1/finance_graph/data_20200204/gap/rel_fund_has_custodian.csv' AS row return row",
    "with row.fund as fund_code,
          row.fullname as company,
          row.type as type
    MERGE (a:FUND {fund_code:fund_code})          
    MERGE (b:COMPANY {company:company})
    MERGE (a)-[r:HAS_CUSTODIAN]->(b)
    RETURN a,b,r
    ",
    {batchSize:1000, parallel:true})
    """

In [117]:
result = graph.run(query_rel_fund_has_custodian)

In [118]:
check_result(result)

<Record batches=1 total=0 timeTaken=0 committedOperations=0 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 1, 'committed': 1, 'failed': 0, 'errors': {}} operations={'total': 0, 'committed': 0, 'failed': 0, 'errors': {}} wasTerminated=False>


### 更新关系：公募基金 --> 管理人

In [119]:
query_rel_fund_has_management = """
CALL apoc.periodic.iterate(
    "LOAD CSV WITH HEADERS FROM 'file:///media/kevin/storage1/finance_graph/data_20200204/gap/rel_fund_has_management.csv' AS row return row",
    "with row.fund as fund_code,
          row.fullname as company,
          row.type as type
    MERGE (a:FUND {fund_code:fund_code})          
    MERGE (b:COMPANY {company:company})
    MERGE (a)-[r:HAS_MANAGEMENT]->(b)
    RETURN a,b,r
    ",
    {batchSize:1000, parallel:true})
    """

In [120]:
result = graph.run(query_rel_fund_has_custodian)

In [121]:
check_result(result)

<Record batches=1 total=0 timeTaken=0 committedOperations=0 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 1, 'committed': 1, 'failed': 0, 'errors': {}} operations={'total': 0, 'committed': 0, 'failed': 0, 'errors': {}} wasTerminated=False>


### 更新关系：公募基金持仓数据

In [122]:

query_rel_fund_listed_company_portfolio = """
CALL apoc.periodic.iterate(
    "LOAD CSV WITH HEADERS FROM 'file:///media/kevin/storage1/finance_graph/data_20200204/gap/rel_fund_listed_company_portfolio.csv' AS row return row",
    "with row.fund as fund_code,
          row.fullname as company,
          row.ann_date as ann_date,
          row.end_date as end_date,
          row.mkv as mkv,
          row.amount as amount,
          row.stk_mkv_ratio as stk_mkv_ratio,
          row.stk_float_ratio as stk_float_ratio,
          row.type as type
    MERGE (a:FUND {fund_code:fund_code})          
    MERGE (b:COMPANY {company:company})
    MERGE (a)-[r:IN_PORTFOLIO]->(b)
        SET r.ann_date = (CASE WHEN ann_date <> 'NotChange' THEN toFloat(ann_date) ELSE r.ann_date END)
        SET r.end_date = (CASE WHEN end_date <> 'NotChange' THEN toFloat(end_date) ELSE r.end_date END)
        SET r.mkv = (CASE WHEN mkv <> 'NotChange' THEN toFloat(mkv) ELSE r.mkv END)        
        SET r.amount = (CASE WHEN amount <> 'NotChange' THEN toFloat(amount) ELSE r.amount END)
        SET r.stk_mkv_ratio = (CASE WHEN stk_mkv_ratio <> 'NotChange' THEN toFloat(stk_mkv_ratio) ELSE r.stk_mkv_ratio END)        
        SET r.stk_float_ratio = (CASE WHEN stk_float_ratio <> 'NotChange' THEN toFloat(stk_float_ratio) ELSE r.stk_float_ratio END)        
    RETURN a,b,r
    ",
    {batchSize:500, parallel:false})
    // 取消并行
    """

In [123]:
result = graph.run(query_rel_fund_listed_company_portfolio)

In [124]:
check_result(result)

<Record batches=110 total=54559 timeTaken=9 committedOperations=54559 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 110, 'committed': 110, 'failed': 0, 'errors': {}} operations={'total': 54559, 'committed': 54559, 'failed': 0, 'errors': {}} wasTerminated=False>


### 更新结果检查

针对下面两条记录会在更新前后做对比

|fund|fullname|ann_date|end_date|mkv|amount|stk_mkv_ratio|stk_float_ratio|type|
|--|--|--|--|--|--|--|--|--|
|000001|南极电商股份有限公司|20200121|20191231|149832015.87|NotChange|4.31|NotChange|IN_PORTFOLIO|
|000001|保利联合化工控股集团股份有限公司|20130328|20121231|8927779.35|363657.0|0.15|0.21|IN_PORTFOLIO|

#### 更新关系

对于000001（华夏成长基金）持仓 南极电商股份有限公司 的信息。在 20200121 有更新。分别对 持有股票市值（mkv）和 占股票市值比 （stk_mkv_ratio）发生变化。

![](../pictures/update_fund_combine1.png)

#### 新增关系

对于000001（华夏成长基金）持仓 保利联合化工控股集团股份有限公司 的信息。

![](../pictures/update_fund_combine2.png)

通过前后对比，可以发现，成功完成了图谱数据的更新。

在实际工作时，可以结合具体的业务需要调整更新频次，每天/每周/每月。